
# Building Multi-Agent Systems with Strands Agent Graph
Multi-agent systems leverage multiple specialized AI agents working together to solve complex problems through coordinated collaboration. Each agent has specific capabilities and roles, connected through explicit communication pathways.

In this lab, you'll learn to build multi-agent systems using the Strands Agent SDK. We'll progress from basic concepts to advanced implementations, exploring different topologies and real-world applications.

**Learning Objectives:**
By the end of this notebook, you'll be able to:
- Understand the three core components of agent graphs (nodes, edges, conditions)
- Send targeted messages between specific agents
- Monitor and control multi-agent networks
- Design specialized agent systems for real-world scenarios

## Prerequisites

- Python 3.10+
- AWS account with Anthropic Claude 3.7 enabled on Amazon Bedrock
- IAM role with permissions to use Amazon Bedrock
- Basic understanding of AI agents and prompt engineering

## Setup and Installation

Before we start, let's install the requirement packages for `strands-agents` and `strands-agents-tools`

In [6]:
!pip install -r requirements.txt

### Importing required packages

Next we can import the required packages

In [7]:
from strands import Agent

## Understanding Agent Graph Components
An agent graph is a structured network of interconnected AI agents designed to solve complex problems through coordinated collaboration. Each agent represents a specialized node with specific capabilities, and the connections between agents define explicit communication pathways.

Before we start building, let's understand the three primary components of an agent graph:

### 1. Nodes (Agents)
Each node represents an AI agent with:
- **Identity**: Unique identifier within the graph
- **Role**: Specialized function or purpose
- **System Prompt**: Instructions defining the agent's behavior
- **Tools**: Capabilities available to the agent

### 2. Edges (Connections)
Edges define communication pathways with:
- **Direction**: One-way or bidirectional information flow
- **Condition**: Optional function that determines if the edge should be traversed
- **Dependencies**: Define execution order and data flow between nodes

### 3. GraphBuilder
The GraphBuilder provides a simple interface for constructing graphs:
- **add_node()**: Add an agent or multi-agent system as a node
- **add_edge()**: Create a dependency between nodes
- **set_entry_point()**: Define starting nodes for execution
- **build()**: Validate and create the Graph instance



### Basic processing

Let's start with a simple example of one task processed by two different agents providing an output that will depend on their defined role. Take a look at the execution order of the nodes and also the fact that with STrands SDK you can explicitly get a response only from one single node if needed. Architecture looks as following:

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [8]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

# Create specialized agents
coordinator = Agent(name="coordinator", system_prompt="You are a research team leader coordinating specialists. Provide a short analysis, no need for follow ups")
analyst = Agent(name="data_analyst", system_prompt="You are a data analyst specializing in statistical analysis. Provide a short analysis, no need for follow ups")
domain_expert = Agent(name="domain_expert", system_prompt="You are a domain expert with deep subject knowledge. Provide a short analysis, no need for follow ups")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# Add edges (dependencies)
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("team_lead")

# Build the graph
graph = builder.build()

#Execute task on newly built graph
result = graph("Analyze the impact of remote work on employee productivity.Provide a short analysis, no need for follow ups")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes
print("\n")
print("=============Expert node results only:======================")
print("============================================================")
print(result.results["expert"].result)

Graph without execution limits may run indefinitely if cycles exist


## Remote Work Productivity Impact Analysis

**Key Findings:**

**Positive Impacts:**
- **Increased flexibility** leads to better work-life balance, reducing burnout and improving focus
- **Elimination of commute time** adds 1-2 productive hours to employees' days
- **Reduced workplace distractions** (meetings, interruptions) allows for deeper concentration
- **Personalized work environments** enable employees to optimize their productivity settings

**Negative Impacts:**
- **Communication delays** slow down collaborative projects and decision-making
- **Home distractions** (family, household tasks) can fragment attention
- **Technology barriers** create inefficiencies for some workers
- **Isolation effects** may reduce motivation and engagement over time

**Net Assessment:**
Research indicates a **10-15% overall productivity increase** for remote workers, with significant variation by role type. Knowledge workers and individual contributors see the greatest gains, while collaborative 

### Parallel processing

Now let's create a topology when we will have 2 agents processing the request looking at 2 different aspect  of the problem and have them input into a final agent responsible for summarization and risk calculation based on provided input 
<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [9]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

financial_advisor = Agent(name="financial_advisor", system_prompt="You are a financial advisor focused on cost-benefit analysis, budget implications, and ROI calculations. Engage with other experts to build comprehensive financial perspectives.")
technical_architect = Agent(name="technical_architect", system_prompt="You are a technical architect who evaluates feasibility, implementation challenges, and technical risks. Collaborate with other experts to ensure technical viability.")
market_researcher = Agent(name="market_researcher", system_prompt="You are a market researcher who analyzes market conditions, user needs, and competitive landscape. Work with other experts to validate market opportunities.")
risk_analyst = Agent(name="risk_analyst", system_prompt="You are a risk analyst who identifies potential risks, mitigation strategies, and compliance issues. Collaborate with other experts to ensure comprehensive risk assessment.")


# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# Add edges (dependencies)
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("finance_expert")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# Get results from specific nodes

print("Financial Advisor:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("Technical Expert:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("Market Researcher:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

Graph without execution limits may run indefinitely if cycles exist


Based on the initial data, this appears to be a strong investment opportunity, but I need to dig deeper into the financial structure to provide a comprehensive assessment.

## Key Financial Metrics Analysis

**ROI Calculation:**
- 150% ROI over 3 years = $3M total return on $2M investment
- This translates to approximately 35% annualized return, which is excellent for operational technology investments

## Critical Questions for Complete Assessment:

**Cash Flow Structure:**
- What's the projected annual cash flow breakdown? Front-loaded or gradual?
- When does the platform reach break-even point?

**Cost Structure Deep-dive:**
- Is the $2M purely development/implementation, or does it include 3-year operational costs?
- What are ongoing costs (licensing, maintenance, staff training, integration)?

**Revenue Impact Validation:**
- How was the 150% ROI calculated? Cost savings, revenue increase, or combination?
- What's the baseline for current customer service costs we're comparing aga

### Branching with conditions

Let's create an agent graph that would classify the request and depending on conditions we define in the code - will route the request either to technical or business agent.

Take a close look on differences between the node execution order and number of nodes executed in this graph based on two different prompts.

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>

In [11]:
#Initialize an agent with agent_graph capability
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# Create specialized agents

classifier = Agent(name="classifier", system_prompt="You are an agent responsible for classification of the report request, return only Technical or Business clasification.")
technical_report = Agent(name="technical_expert", system_prompt="You are a technical expert htat focuses on providing short summary from technical perspective")
business_report = Agent(name="business_expert", system_prompt="You are a business expert that focuses on providing short summary from business perspective")

# Build the graph
builder = GraphBuilder()

# Add nodes
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# Add edges (dependencies)
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# Set entry points (optional - will be auto-detected if not specified)
builder.set_entry_point("classifier")

# Build the graph
graph = builder.build()

print("============================================================")
print("============================================================")

#Execute task on newly built graph
result = graph("Provide report on technical aspect of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

#Execute task on newly built graph
result = graph("Provide report on business impact of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============Node execution order:==========================")
print("============================================================")

# See which nodes were executed and in what order
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============Graph metrics:=================================")
print("============================================================")


# Get performance metrics
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# Get results from specific nodes

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

Graph without execution limits may run indefinitely if cycles exist


Technical# Technical Aspects of Working From Home: Key Considerations and Risk Factors

## Core Technical Requirements

**Essential Infrastructure:**
- Reliable high-speed internet (minimum 25 Mbps download, 10 Mbps upload)
- Backup connectivity solutions (mobile hotspot, secondary ISP)
- Adequate hardware: laptop/desktop with sufficient RAM (8GB+) and processing power
- Proper display setup and ergonomic peripherals

## Security Risk Factors

**Network Vulnerabilities:**
- Unsecured home Wi-Fi networks
- Shared network access with family devices
- Lack of enterprise-grade firewall protection

**Data Protection Concerns:**
- Inadequate endpoint security
- Unencrypted data transmission
- Physical security of devices and documents
- BYOD policy compliance gaps

## Technical Challenges

**Connectivity Issues:**
- Internet service reliability and bandwidth limitations
- VPN performance degradation
- Latency impacts on real-time collaboration

**IT Support Limitations:**
- Remote troublesho

## Key Takeaways and Best Practices


### Best Practices:

**Design for acyclicity:** Ensure your graph has no cycles</p>
**Use meaningful node IDs:** Choose descriptive names for nodes</p>
**Validate graph structure:** The builder will check for cycles and validate entry points</p>
**Handle node failures:** Consider how failures in one node affect the overall workflow</p>
**Use conditional edges:** For dynamic workflows based on intermediate results</p>
**Consider parallelism:** Independent branches can execute concurrently</p>
**Nest multi-agent patterns:** Use Swarms within Graphs for complex workflows</p>
**Leverage multi-modal inputs:** Use ContentBlocks for rich inputs including images</p>

## Conclusion

You've now mastered the fundamentals of building multi-agent systems with Strands Agent Graph! You can create sophisticated networks of specialized AI agents that collaborate to solve complex problems.

The key to successful multi-agent systems is:
- Matching topology to use case
- Defining clear agent roles and responsibilities  
- Establishing proper communication patterns
- Managing resources and cleanup effectively

From here, you can build increasingly sophisticated systems for real-world applications in research, content creation, decision-making, customer service, and beyond.